# AUTO$^2$ and AUTO-Demos : *lrz* - The Lorenz equation.

**This is an example on how to use AUTO$^2$ to explore the AUTO Demos *lrz* bifurcations and solutions.**

This example is concerned with the famous [Lorenz 63](https://en.wikipedia.org/wiki/Lorenz_system) model for [atmospheric convection](https://en.wikipedia.org/wiki/Lorenz_system#Model_for_atmospheric_convection), depicting the famous "butterfly wings".

![ButterflyUrl](https://upload.wikimedia.org/wikipedia/commons/1/13/A_Trajectory_Through_Phase_Space_in_a_Lorenz_Attractor.gif "butterfly")
Source: Wikipedia Commons

Here, one can see a sample solution on the attractor when $\rho = 28$, $\sigma = 10$, and $\beta = 8/3$ (see model equations below).

The equations of the model are given by:

$$
\dot x = \sigma (y-x)
$$
$$
\dot y = x \rho - y - x z
$$
$$
\dot z = x y - \beta z
$$

The free parameter in the current example notebook is $\rho$ . Other parameters are fixed.

We are thus going to find the fixed points and periodic orbits of this system and continue them by varying $\rho$.

#### References

* Lorenz, Edward Norton (1963). "Deterministic nonperiodic flow". *Journal of the Atmospheric Sciences*. **20** (2): 130–141. [doi:10.1175/1520-0469(1963)020<0130:DNF>2.0.CO;2](https://doi.org/10.1175%2F1520-0469%281963%29020%3C0130%3ADNF%3E2.0.CO%3B2).
* Barrio, R., Shilnikov, A., & Shilnikov, L. (2012). Kneadings, symbolic dynamics and painting Lorenz chaos. *International Journal of Bifurcation and Chaos*, **22**(04), 1230016. [doi:10.1142/S0218127412300169](https://doi.org/10.1142/S0218127412300169)

## Code

First we set the Python path if needed:

In [ ]:
import sys, os

In [ ]:
sys.path.extend([os.path.abspath('../../../')])

And load the needed libraries, including AUTO$^2$:

In [ ]:
import numpy as np
from numba import njit
from scipy.optimize import root
from scipy.integrate import solve_ivp

In [ ]:
from auto2.diagrams.bifurcations import BifurcationDiagram

Creating the model equations

In [ ]:
@njit
def lrz(X, rho, beta, sigma):
    x = X[0]
    y = X[1]
    z = X[2]
    F = np.zeros(3)
    F[0] = sigma * (y - x)
    F[1] = rho * x - y - x * z
    F[2] = x * y - beta * z
    return F

@njit
def lrzt(t, X, rho, beta, sigma):
    return lrz(X, rho, beta, sigma)

and define a set of standard parameters:

In [ ]:
params = {
    'rho': 24.5,
    'beta': 8./3,
    'sigma': 10.,
}

For reference later, we can compute a long trajectory on the attractor of this model:

In [ ]:
#first a transient
ic = np.zeros(3) + 0.01
transient = solve_ivp(lrzt, (0., 1000.), ic, args=tuple(params.values()))

In [ ]:
# then the trajectory itself
ic = transient['y'][:, -1]
trajectory = solve_ivp(lrzt, (0., 1000.), ic, args=tuple(params.values()))

Finding all the fixed points of the Lorenz 63 system for $\beta = 8/3$, $\sigma = 10$ and $\rho = 0$ :

In [ ]:
params['rho'] = 0.
nsearch = 1000

# Start on random initial conditions
ic = 2 * (np.random.rand(nsearch, 3) - 0.5) * 10.

eps = 1.e-6
fixed_points = dict()

sol_idx = 1
for i in range(nsearch):
    sol = root(lrz, ic[i, :], args=tuple(params.values()))
    if sol.success:
        for idx in fixed_points:
            if np.linalg.norm(fixed_points[idx] - sol.x) < eps:
                break
        else:
            fixed_points[sol_idx] = sol.x
            sol_idx+=1


We have now the list of fixed points `fixed_points` and parameters dictionnary `params` that AUTO$^2$ will have to continue

In [ ]:
initial_points = list()

for p in fixed_points:
    initial_points.append({'parameters': params, 'initial_data': fixed_points[p]})


and thus we are now ready to compute the diagram of fixed points as a function of $\rho$. Note that we specify that the bifurcation diagram object must load the ̀`lrz.f90` and `c.lrz` files where the Lorenz 63 model equations and continuation parameters have been written:

In [ ]:
b = BifurcationDiagram('lrz')

b.compute_fixed_points_diagram(initial_points,extra_comparison_parameters=['x', 'y'], comparison_tol=[1.e-1] * 3,
                               ICP=['rho'], NMX=300, UZSTOP={'rho':[-10.,40.]}, UZR={'rho': list(np.arange(2,30, 2.5))}, NPR=0)

We can now plot the result as functions of $\rho$ and $L^2$ norm :

In [ ]:
b.plot_fixed_points_diagram();

and also as functions of $x$ and $y$ :

In [ ]:
b.plot_fixed_points_diagram((2,3));

or in 3D as functions of $\rho$, $L^2$ norm and $x$ :

In [ ]:
b.plot_fixed_points_diagram_3D();

We see that at 2 branches were found. The last one is symmetric and present distinct Hopf bifurcations.

We can continue periodic orbits out of these Hopf bifurcations : 

In [ ]:
b.compute_periodic_orbits_diagram(3, extra_comparison_parameters=['x', 'y'], max_number_bp=None, comparison_tol=[1.e-3, 1.e-3, 1.e-3],
                                  ICP=['rho'])


and plot the results on a bifurcation diagram:

In [ ]:
ax = b.plot_fixed_points_diagram()
b.plot_periodic_orbits_diagram(ax=ax, cmap='gist_ncar');

We can also plot both the bifurcation diagram and the solutions for a given value of $\rho$:

In [ ]:
b.plot_diagram_and_solutions(22., solutions_variables=(0, 1), fixed_points_diagram_kwargs={'legend': True}, 
                             periodic_orbits_diagram_kwargs={'cmap': 'gist_ncar'});

You can compare this result with the animated figure at the top of the notebook.
We can also plot the result in 3 dimensions to get a better view:

In [ ]:
b.plot_diagram_in_3D_and_solutions_in_3D(22., solutions_variables=(0, 1, 2), fixed_points_diagram_kwargs={}, 
                                         periodic_orbits_diagram_kwargs={'cmap': 'gist_ncar'});

Finally, it is not hard to also plot the dynamics on the attractor (represented by the long trajectory computed beforehand) on top of the solutions to see their relevance:

In [ ]:
axs = b.plot_diagram_and_solutions(24.5, solutions_variables=(0, 1), fixed_points_diagram_kwargs={'legend': True}, 
                             periodic_orbits_diagram_kwargs={'cmap': 'gist_ncar'})
axs[1].plot(trajectory['y'][0], trajectory['y'][1], marker='o', ms=0.07, ls='', color='darkgray')

One can see that the chaotic dynamics is constrained by the two found periodic orbits. In fact, for $\rho=24.5$ (and values around), the basin of the Lorenz attractor is shielded away from the two stable symmetric equilibrium states by the 2D cylinder-shaped stable manifolds of the two “threshold” saddle orbits seen here. For more information, see Section 2.2 of Barrio et al. (2012) (see reference at the top of the notebook).